### Structure of DataFrames
* **Provences**: [Provence, Domain, Regent, Terrain, Loyalty, Taxation, Population, Magic, Castle, Capital, Position, Troops]
* **Holdings**: [Provence, Domain, Regent, Type, Level]
* **Regents**: [Regent, Full Name, Player, Class, Level, Alignment, Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion, Regency Points, Gold Bars, Regency Bonus, Attitude, Lieutenants]
* **Geography**: [Provence, Neighbor, Border, Road, Caravan, Shipping]
* **Relationship**: [Regent, Other, Diplomacy, Payment, Vassalage]
* **Troops**: [Regent, Provence, Type, Cost, CR]

* **Seasons**: A dctionary of season-dataframes (to keep track of waht happened)



In [1]:
from regency import Regency
from DQNAgent import DQNAgent
from IPython.display import clear_output

Game = Regency()
for a in range(10):
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs()

clear_output()
Game.Seasons[9]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,LPA,False,No Event,140,22,18,9,10,Rich
1,DW,False,Corruption/Crime,230,19,75,17,10,Rich
2,GH,False,No Event,110,19,20,9,9,Rich
3,CSH,False,Corruption/Crime,110,18,18,9,9,Rich
4,Sw2,False,No Event,180,18,0,0,0,Dormant
5,IHH,False,No Event,70,17,7,6,6,Average
6,Sw,False,No Event,120,17,0,0,0,Dormant
7,WM,False,Monsters,80,16,0,2,3,Bare
8,TCV,False,No Event,20,13,0,1,1,Dormant
9,HA,False,No Event,40,12,1,2,1,Dormant


In [5]:
Game.Provences

,Provence,Domain,Regent,Terrain,Loyalty,Taxation,Population,Magic,Castle,Capital,Position
0,Bogsend,Coeranys,EC,Swamp,Average,Moderate,3,2,0,False,"[1543, 1985]"
1,Caudraight,Coeranys,EC,Plains,Average,Moderate,4,1,0,False,"[1556, 1914]"
2,Deepshadow,Coeranys,EC,Plains,Average,Moderate,3,2,0,False,"[1562, 1817]"
3,Duornil,Coeranys,EC,Plains,Average,Moderate,2,3,0,False,"[1606, 1837]"
4,Mistil,Coeranys,EC,Plains,Average,Moderate,2,3,0,False,"[1662, 1966]"
5,Ranien,Coeranys,EC,Plains,Average,Moderate,2,3,0,False,"[1610, 1921]"
6,Ruorven,Coeranys,EC,Swamp,Average,Moderate,4,1,0,True,"[1610, 2043]"
7,Algael,Osoerde,JR,Swamp,Average,Severe,2,3,0,False,"[1355, 2226]"
8,Brothendar,Osoerde,JR,Plains,Average,Severe,3,2,0,False,"[1377, 2177]"
9,Ghalliere,Osoerde,JR,Plains,Average,Severe,2,3,0,False,"[1394, 2110]"


In [3]:
Game.agent['Aggresive'].tax_model

In [6]:
Game.agent['Normal'].epsilon

11